In [ ]:
!pip -q install --upgrade pip
!pip -q uninstall -y pillow -q
!pip -q install "pillow<12.0,>=9.5.0" transformers accelerate bitsandbytes timm einops qwen-vl-utils flask pyngrok

import os
import json
import torch
import re
from dataclasses import dataclass
from typing import List, Dict, Any
from PIL import Image, ImageOps
import contextlib
from flask import Flask, request, jsonify
from pyngrok import ngrok
import base64
import io
from google.colab import userdata # Import userdata to access secrets

# Configuration
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Initialize models
from transformers import AutoProcessor, AutoModelForVision2Seq, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler

# Model configurations
CAPTION_MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
OPT_LLM_ID = "Qwen/Qwen2.5-3B-Instruct"

bnb_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load Vision-Language Model
vl_processor = AutoProcessor.from_pretrained(CAPTION_MODEL_ID, trust_remote_code=True)
try:
    vl_model = AutoModelForVision2Seq.from_pretrained(
        CAPTION_MODEL_ID,
        quantization_config=bnb_4bit,
        dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    ).eval()
except TypeError:
    vl_model = AutoModelForVision2Seq.from_pretrained(
        CAPTION_MODEL_ID,
        quantization_config=bnb_4bit,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    ).eval()

# Load LLM for prompt optimization
opt_tokenizer = AutoTokenizer.from_pretrained(OPT_LLM_ID)
opt_model = AutoModelForCausalLM.from_pretrained(
    OPT_LLM_ID,
    quantization_config=bnb_4bit,
    torch_dtype=torch.bfloat16,
    device_map="auto",
).eval()

@dataclass
class SceneRecord:
    file: str
    caption: str
    camera: Dict[str, Any]
    time: str
    lighting: str
    mood: str
    environment: str
    objects: List[str]
    actions: List[str]
    composition: List[str]
    colors: List[str]
    tags: List[str]

class MovieSceneGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        dtype = torch.float16 if self.device == "cuda" else torch.float32

        self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id, torch_dtype=dtype, safety_checker=None, requires_safety_checker=False
        )
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(
            self.pipe.scheduler.config, use_karras_sigmas=True,
            algorithm_type="dpmsolver++", solver_type="midpoint",
        )
        self.pipe.enable_attention_slicing()
        self.pipe.enable_vae_slicing()
        self.pipe.enable_vae_tiling()
        if torch.cuda.is_available():
            try:
                self.pipe.enable_model_cpu_offload()
            except Exception:
                self.pipe.enable_sequential_cpu_offload()
        else:
            self.pipe.to("cpu")

    def fit_to_box(self, img, w=512, h=384):
        img = ImageOps.exif_transpose(img).convert("RGB")
        ar_img, ar_tar = img.width / img.height, w / h
        if ar_img > ar_tar:
            new_h, new_w = h, int(round(h * ar_img))
        else:
            new_w, new_h = w, int(round(w / ar_img))
        img = img.resize((new_w, new_h), Image.LANCZOS)
        left, top = (new_w - w)//2, (new_h - h)//2
        return img.crop((left, top, left + w, top + h))

# Helper functions
def vl_generate_json_caption(img):
    messages = [
        {"role": "system", "content": "You are a film-savvy visual describer. Be precise and concise."},
        {"role": "user", "content": [
            {"type": "image", "image": img},
            {"type": "text", "text":
             ("Describe this frame for a filmmaker. Respond ONLY in compact JSON with keys: "
              "caption, camera, time, lighting, mood, environment, objects, actions, "
              "composition, colors, tags.")}
        ]}
    ]

    image_inputs, video_inputs = process_vision_info(messages)
    text = vl_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = vl_processor(text=[text], images=image_inputs, videos=video_inputs, return_tensors="pt").to(vl_model.device)

    with torch.inference_mode():
        out_ids = vl_model.generate(**inputs, max_new_tokens=320, do_sample=False, temperature=0.0)
    out_txt = vl_processor.batch_decode(out_ids, skip_special_tokens=True)[0]

    start, end = out_txt.find("{"), out_txt.rfind("}")
    blob = out_txt[start:end+1] if start != -1 and end != -1 else "{}"
    try:
        data = json.loads(blob)
    except json.JSONDecodeError:
        data = {"caption": out_txt.strip()}
    return data

def normalize_scene(meta: Dict[str, Any]) -> SceneRecord:
    return SceneRecord(
        file=meta.get("file", ""),
        caption=meta.get("caption", ""),
        camera=meta.get("camera", {}),
        time=meta.get("time", ""),
        lighting=meta.get("lighting", ""),
        mood=meta.get("mood", ""),
        environment=meta.get("environment", ""),
        objects=list(meta.get("objects", []))[:10],
        actions=list(meta.get("actions", []))[:10],
        composition=list(meta.get("composition", []))[:8],
        colors=list(meta.get("colors", []))[:6],
        tags=list(meta.get("tags", []))[:15],
    )

def chat_complete(model, tokenizer, system_msg: str, user_msg: str, max_new_tokens=512, temperature=0.0):
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg + "\n\nReturn STRICT JSON only. Start with '{' and end with '}'."}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=temperature > 0.0,
            temperature=temperature,
            top_p=0.9 if temperature > 0.0 else None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.05
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def extract_json(text: str):
    text = text.strip().strip("`")
    m = re.search(r"\{.*\}", text, flags=re.S)
    if not m:
        return None
    blob = m.group(0)
    blob = re.sub(r",\s*([}\]])", r"\1", blob)
    try:
        return json.loads(blob)
    except json.JSONDecodeError:
        return None

PROMPT_SYSTEM = (
    "You are a prompt engineer for image diffusion models. "
    "Given (A) a filmmaker's goal and (B) factual scene descriptors, "
    "produce SD-ready prompts as JSON only."
)

def build_optimizer_prompt(user_goal: str, scene: SceneRecord) -> str:
    scene_json = json.dumps(scene.__dict__, ensure_ascii=False)
    return (
        "(A) Filmmaker goal:\n" + user_goal + "\n\n"
        "(B) Scene facts (JSON):\n" + scene_json + "\n\n"
        "Rules:\n"
        "- Keep geometry/environment consistent with (B) unless (A) requires changes.\n"
        "- Include 1–3 hero objects/actions from (B) that support (A).\n"
        "- Output JSON with keys: "
        "variants (array of {name, prompt}), negative_prompt, notes.\n"
        "- Provide exactly three variants: creative, balanced, strict.\n"
    )

def optimize_prompts(user_goal: str, scene: SceneRecord):
    user_msg = build_optimizer_prompt(user_goal, scene)
    text = chat_complete(opt_model, opt_tokenizer, PROMPT_SYSTEM, user_msg, max_new_tokens=640, temperature=0.0)
    js = extract_json(text)
    return js if js is not None else {"raw": text}

def choose_prompt(block, prefer="balanced"):
    variants = block.get("variants", [])
    neg = block.get("negative_prompt", "text, watermark, logo, low quality, blurry, deformed, extra limbs")
    if variants:
        for v in variants:
            if v.get("name","").lower().startswith(prefer):
                return v["prompt"], neg
        return variants[1]["prompt"], neg if len(variants) > 1 else variants[0]["prompt"], neg
    return block.get("raw",""), neg

def base64_to_image(base64_string):
    image_data = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(image_data))

def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Initialize the generator
generator = MovieSceneGenerator()

# Flask API
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "healthy", "device": device})

@app.route('/generate-scene', methods=['POST'])
def generate_scene():
    try:
        data = request.json

        # Required parameters
        if 'image' not in data or 'user_goal' not in data:
            return jsonify({"error": "Missing required parameters: 'image' and 'user_goal'"}), 400

        # Optional parameters with defaults
        seed = data.get('seed', 42)
        strength = data.get('strength', 0.47)
        guidance = data.get('guidance', 20)
        steps = data.get('steps', 40)
        variant = data.get('variant', 'strict')

        # Convert base64 image to PIL Image
        input_image = base64_to_image(data['image'])
        user_goal = data['user_goal']

        # Step 1: Generate scene description
        scene_meta = vl_generate_json_caption(input_image)
        scene = normalize_scene(scene_meta)

        # Step 2: Optimize prompts
        opt_result = optimize_prompts(user_goal, scene)
        full_prompt, neg_prompt = choose_prompt(opt_result, prefer=variant)

        # Step 3: Generate new image
        init_image = generator.fit_to_box(input_image, 512, 384)
        torch_generator = torch.Generator("cpu").manual_seed(seed)

        ctx = torch.autocast("cuda") if generator.device == "cuda" else contextlib.nullcontext()
        with ctx:
            result = generator.pipe(
                prompt=full_prompt,
                negative_prompt=neg_prompt,
                image=init_image,
                strength=strength,
                guidance_scale=guidance,
                num_inference_steps=steps,
                generator=torch_generator,
                num_images_per_prompt=1
            )

        output_image = result.images[0]
        output_base64 = image_to_base64(output_image)

        return jsonify({
            "success": True,
            "output_image": output_base64,
            "scene_description": scene.__dict__,
            "optimized_prompt": full_prompt,
            "negative_prompt": neg_prompt,
            "parameters": {
                "seed": seed,
                "strength": strength,
                "guidance": guidance,
                "steps": steps,
                "variant": variant
            }
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/describe-scene', methods=['POST'])
def describe_scene():
    try:
        data = request.json
        if 'image' not in data:
            return jsonify({"error": "Missing required parameter: 'image'"}), 400

        input_image = base64_to_image(data['image'])
        scene_meta = vl_generate_json_caption(input_image)
        scene = normalize_scene(scene_meta)

        return jsonify({
            "success": True,
            "scene_description": scene.__dict__
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start the server
if __name__ == "__main__":
    print("Starting Scene Generator API...")
    print("Loading models... This may take a few minutes.")

    # Test that models are loaded
    print("✓ Models loaded successfully!")

    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(5000)
        print(f"\n🌐 Public URL: {public_url}")
        print("📡 API Endpoints:")
        print(f"  Health Check: {public_url}/health")
        print(f"  Generate Scene: {public_url}/generate-scene")
        print(f"  Describe Scene: {public_url}/describe-scene")
    else:
        print("\nNGROK_AUTH_TOKEN not found in secrets. Please add it to run the API.")


    # Run the Flask app
    app.run(host='0.0.0.0', port=5000, debug=False)

Device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Starting Scene Generator API...
Loading models... This may take a few minutes.
✓ Models loaded successfully!

🌐 Public URL: NgrokTunnel: "https://kamdyn-reconstructed-mignon.ngrok-free.dev" -> "http://localhost:5000"
📡 API Endpoints:
  Health Check: NgrokTunnel: "https://kamdyn-reconstructed-mignon.ngrok-free.dev" -> "http://localhost:5000"/health
  Generate Scene: NgrokTunnel: "https://kamdyn-reconstructed-mignon.ngrok-free.dev" -> "http://localhost:5000"/generate-scene
  Describe Scene: NgrokTunnel: "https://kamdyn-reconstructed-mignon.ngrok-free.dev" -> "http://localhost:5000"/describe-scene
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
Token indices sequence length is longer than the specified maximum sequence length for this model (1077 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['", " caption ": " a bustling city street with tall buildings and parked cars , illuminated by sunlight .", " camera ": " wide shot ", " time ": " daytime ", " lighting ": " natural sunlight ", " mood ": " busy and vibrant ", " environment ": " urban setting ", " objects ": [" c ", " a ", " r ", " s ", ",", " ", " b ", " u ", " i ", " l "], " actions ": [" p ", " e ", " o ", " p ", " l ", " e ", " ", " w ", " a ", " l "], " compos

  0%|          | 0/18 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [18/Oct/2025 18:26:46] "POST /generate-scene HTTP/1.1" 200 -
